<a href="https://colab.research.google.com/github/szymbor2/ATMS597-Project5-GroupG/blob/master/project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b> Imports </b>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import glob
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression

YOUR_DIRECTORY = '/content/drive/My Drive/Colab Notebooks/ATMS597/project5/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<b> Get Data </b>

In [0]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-{2000..2019}/64010KSTC{2000..2019}{01..12}.dat
!cp *.dat YOUR_DIRECTORY + 'project5_data/'

--2020-04-16 19:19:02--  ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KSCT200001.dat
           => ‘64010KSCT200001.dat’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.101, 2610:20:8040:2::101
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.101|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/asos-fivemin/6401-2000 ... done.
==> SIZE 64010KSCT200001.dat ... done.

==> PASV ... done.    ==> RETR 64010KSCT200001.dat ... 
No such file ‘64010KSCT200001.dat’.



In [0]:
import glob
import pandas as pd
import numpy as np
import datetime

files = sorted(glob.glob('/content/drive/My Drive/project5_data/*.dat'))
df = pd.DataFrame([line.strip().split(' ') for line in open(files[0], 'r')])
del df[3]
n_cols = len(df.columns)
columns = ['StnNo', 'ContainsDate', 'Time', "Interval", 'Sttn', 'UTC', 'Type', 'Wind', "Weath/Obstr", 'SkyCond', 'Temp',
              'n_days', 'MSLP', 'RelHum', ' ', 'WndDir/Spd', 'WindMagnetic', 'RMK', 'AO2']
cols_already = len(columns)
cols_new = []
cols_new = [np.append(cols_new,'X') for i in range(n_cols-cols_already)]
columns = np.append(columns, cols_new)
df.columns = columns

y = 2000
for i in range(np.shape(df)[0]-1, np.shape(df)[0]): #np.shape(df)[0]
    day = df['ContainsDate'][i][-8:] 
    time = df['Time'][i]
    date = datetime.datetime.strptime(day+' '+time, '%m/%d/%y %H:%M:%S')
    (h, m, s) = time.split(':')
    timeofday = float(int(h) * 3600 + int(m) * 60 + int(s))/(3600*24)
    print(date)
    print(timeofday)
    Gusts = 0
    Wind = 0
    Variable_Winds = 0
    if "KT" in df['Wind'][i]:
        Wind = 1
        Wind_Direction = int(df['Wind'][i][0:3]) # in tens of degrees from true north
        if df['Wind'][i][0:3] == 'VRB':
            Variable_Winds = 1
        else:
            Variable_Winds = 0
        Wind_Speed = int(df['Wind'][i][3:5]) # in whole knots (two digits)
        if df['Wind'][i][5] == 'G':
            Gusts = 1
            Gust_Speed = int(df['Wind'][i][6:8]) # speed in whole knots (two digits)
    else:
        Wind = 0
    print('Wind, Wind_Direction, Wind_Speed, Gusts, Variable_Winds:')
    print(Wind, Wind_Direction, Wind_Speed, Gusts, Variable_Winds)
    Temp = 0
    if "/" in df['Temp'][i]:
        Temp = 1
        Temperature = df['Temp'][i].split('/')[0]
        if 'M' in Temperature:
          Temperature = int(Temperature[1:3])
        else:
          Temperature = int(Temperature)
        Dewpoint = df['Temp'][i].split('/')[1]
        if 'M' in Dewpoint:
          Dewpoint = int(Dewpoint[1:3])
        else:
          Dewpoint = int(Dewpoint) 
    else:
        Temp = 0  
    rhum = df['RelHum'][i]    
    print('Temp, Temperature, Dewpoint, rhum:') 
    print(Temp, Temperature, Dewpoint, rhum) 
    
df

In [17]:
from itertools import islice
with open(YOUR_DIRECTORY+'project5_data/64010KSTC201912.dat', 'r') as f:
    head = list(islice(f, 5))
print(head[0])
print(head[1])
print(head[2])
print(head[3])
print(head[4])

14926KSTC STC20191201062513412/01/19 06:25:31  5-MIN KSTC 011225Z AUTO 01013G22KT 5SM -SN BR OVC024 M03/M05 A2964 1290 85 -500 360/13G22 RMK AO2 P0001 T10281050 $

14926KSTC STC20191201063013412/01/19 06:30:31  5-MIN KSTC 011230Z AUTO 36012G21KT 6SM -SN BR OVC024 M03/M05 A2964 1290 85 -500 360/12G21 RMK AO2 P0001 T10281050 $

14926KSTC STC20191201063513112/01/19 06:35:31  5-MIN KSTC 011235Z AUTO 01015G24KT 8SM -SN OVC025 M03/M05 A2965 1290 85 -500 360/15G24 RMK AO2 P0001 T10281050 $

14926KSTC STC20191201064013112/01/19 06:40:31  5-MIN KSTC 011240Z AUTO 01013G24KT 8SM -SN OVC025 M03/M06 A2965 1280 81 -500 010/13G24 RMK AO2 P0001 T10281056 $

14926KSTC STC20191201064513112/01/19 06:45:31  5-MIN KSTC 011245Z AUTO 01013G23KT 9SM -SN OVC025 M03/M06 A2965 1280 81 -500 010/13G23 RMK AO2 P0001 T10281056 $

